In [1]:
# keller_general.py
import itertools
import networkx as nx
import time
import pickle
import os

# ------------------------------------------------------------
# 1. Vertex encoding utilities (pure bitwise)
# ------------------------------------------------------------
def encode_tuple(t):
    """Encode Keller vertex tuple (each coord 0..3) into compact integer."""
    x = 0
    for i, v in enumerate(t):
        x |= (v & 0b11) << (2*i)
    return x

def generate_vertices(d):
    """Generate all encoded vertices of Keller graph K(d)."""
    return [encode_tuple(t) for t in itertools.product(range(4), repeat=d)]


# ------------------------------------------------------------
# 2. Fast adjacency test (encoded integer → boolean)
# ------------------------------------------------------------
def keller_adj(a, b, d):
    """
    Fast adjacency test for Keller graph.
    Returns True iff:
      - at least 2 coordinates differ
      - at least one differs by 2 modulo 4
    """
    diff = 0
    mod2 = False
    for i in range(d):
        ai = (a >> (2*i)) & 3
        bi = (b >> (2*i)) & 3
        if ai != bi:
            diff += 1
            if ((ai - bi) & 3) == 2:
                mod2 = True
        if diff >= 2 and mod2:
            return True
    return False


# ------------------------------------------------------------
# 3. Compute Kneser codes: list[int]
# ------------------------------------------------------------
def keller_kneser_codes(d):
    """
    Return Kneser representation of Keller-K(d) as list of integers.
    codes[i] = bitmask where bit j == 1 means j is a NON-neighbor of i.
    """
    t0 = time.time()
    verts = generate_vertices(d)
    n = len(verts)

    codes = [0] * n
    edges = 0

    for i in range(n):
        ai = verts[i]
        for j in range(i+1, n):
            aj = verts[j]
            adj = keller_adj(ai, aj, d)
            if adj:
                edges += 1
            else:
                codes[i] |= 1 << j
                codes[j] |= 1 << i

    t1 = time.time()
    stats = {
        "n": n, 
        "edges": edges, 
        "time": t1 - t0,
        "dimension": d,
        "expected_clique": {3: 5, 4: 12, 5: 28, 6: 60, 7: 124}[d]
    }
    return codes, stats


# ------------------------------------------------------------
# 4. Graph utilities (for validation)
# ------------------------------------------------------------
def graph_from_codes(codes):
    """Rebuild networkx graph from kneser mask list."""
    n = len(codes)
    G = nx.Graph()
    G.add_nodes_from(range(n))
    for i in range(n):
        ci = codes[i]
        for j in range(i+1, n):
            # Adjacent if NOT non-neighbors
            if ((ci >> j) & 1) == 0 and ((codes[j] >> i) & 1) == 0:
                G.add_edge(i, j)
    return G

def native_keller_graph(d):
    """Generate networkx graph of Keller K(d)."""
    verts = generate_vertices(d)
    n = len(verts)
    G = nx.Graph()
    G.add_nodes_from(range(n))
    for i in range(n):
        for j in range(i+1, n):
            if keller_adj(verts[i], verts[j], d):
                G.add_edge(i, j)
    return G


# ------------------------------------------------------------
# 5. Main function to generate and save codes for multiple dimensions
# ------------------------------------------------------------
def generate_all_keller_codes(start_dim=3, end_dim=7):
    """Generate and save Keller graph codes for dimensions 3 to 7."""
    
    # Known maximum clique sizes for Keller graphs
    max_cliques = {
        3: 5,   # K(3)
        4: 12,  # K(4)
        5: 28,  # K(5)
        6: 60,  # K(6)
        7: 124  # K(7)
    }
    
    print("=" * 60)
    print(f"Generating Keller graph codes for dimensions {start_dim} to {end_dim}")
    print("=" * 60)
    
    all_stats = {}
    
    for d in range(start_dim, end_dim + 1):
        print(f"\n{'='*40}")
        print(f"Processing Keller K({d})")
        print(f"{'='*40}")
        
        # Generate codes
        codes, stats = keller_kneser_codes(d)
        
        # Save to file
        filename = f'keller_codes_d{d}.data'
        with open(filename, 'wb') as f:
            pickle.dump(codes, f)
        
        print(f"✓ Generated codes for K({d})")
        print(f"  Vertices: {stats['n']:,}")
        print(f"  Edges: {stats['edges']:,}")
        print(f"  Generation time: {stats['time']:.2f} seconds")
        print(f"  Expected max clique: {stats['expected_clique']}")
        print(f"  Saved to: {filename}")
        
        all_stats[d] = stats
    
    # Create a summary file
    with open('keller_stats.txt', 'w') as f:
        f.write("Keller Graph Statistics\n")
        f.write("=" * 40 + "\n")
        for d, stats in all_stats.items():
            f.write(f"\nK({d}):\n")
            f.write(f"  Vertices: {stats['n']:,}\n")
            f.write(f"  Edges: {stats['edges']:,}\n")
            f.write(f"  Dimension: {d}\n")
            f.write(f"  Expected max clique: {stats['expected_clique']}\n")
    
    print(f"\n{'='*60}")
    print("All codes generated successfully!")
    print(f"Summary saved to: keller_stats.txt")
    
    return all_stats


# ------------------------------------------------------------
# 6. Validation function
# ------------------------------------------------------------
def validate_kneser(d, codes=None, do_iso=False):
    """
    Validate the kneser representation:
      - vertex count
      - edge count
      - degree sequences
      - optional isomorphism check (small d only)
    """
    if codes is None:
        filename = f'keller_codes_d{d}.data'
        with open(filename, 'rb') as f:
            codes = pickle.load(f)
    
    print("Building native graph...")
    G_native = native_keller_graph(d)

    print("Building graph from Kneser codes...")
    G_codes = graph_from_codes(codes)

    print("\n--- VALIDATION REPORT ---")
    print(f"Keller K({d})")

    print(f"Vertices native: {G_native.number_of_nodes()} | from codes: {G_codes.number_of_nodes()}")
    print(f"Edges native: {G_native.number_of_edges()} | from codes: {G_codes.number_of_edges()}")

    deg1 = sorted([d for _, d in G_native.degree()])
    deg2 = sorted([d for _, d in G_codes.degree()])

    print("Degree sequence match:", deg1 == deg2)

    if do_iso and d <= 4:  # Only for small dimensions
        print("Running graph isomorphism...")
        iso = nx.is_isomorphic(G_native, G_codes)
        print("Isomorphic:", iso)


# ------------------------------------------------------------
# 7. Main execution
# ------------------------------------------------------------
if __name__ == "__main__":
    # Generate codes for dimensions 3 to 7
    stats = generate_all_keller_codes(3, 7)
    
    # Optional: Validate one dimension
    print("\n" + "="*60)
    print("Validating K(3)...")
    validate_kneser(3, do_iso=True)

Generating Keller graph codes for dimensions 3 to 7

Processing Keller K(3)
✓ Generated codes for K(3)
  Vertices: 64
  Edges: 1,088
  Generation time: 0.00 seconds
  Expected max clique: 5
  Saved to: keller_codes_d3.data

Processing Keller K(4)
✓ Generated codes for K(4)
  Vertices: 256
  Edges: 21,888
  Generation time: 0.08 seconds
  Expected max clique: 12
  Saved to: keller_codes_d4.data

Processing Keller K(5)
✓ Generated codes for K(5)
  Vertices: 1,024
  Edges: 397,312
  Generation time: 1.19 seconds
  Expected max clique: 28
  Saved to: keller_codes_d5.data

Processing Keller K(6)
✓ Generated codes for K(6)
  Vertices: 4,096
  Edges: 6,883,328
  Generation time: 17.94 seconds
  Expected max clique: 60
  Saved to: keller_codes_d6.data

Processing Keller K(7)
✓ Generated codes for K(7)
  Vertices: 16,384
  Edges: 116,244,480
  Generation time: 330.42 seconds
  Expected max clique: 124
  Saved to: keller_codes_d7.data

All codes generated successfully!
Summary saved to: keller_s